In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("eps130_hw5_seismicRefraction_v2.1.ipynb")

# Seismic Refraction

## Introduction

Seismic refraction is a method utilizing first arrival traveltime measurements made from a linear array of sensors. This method makes use of the fact that when downward propagating seismic waves interact with deeper and faster layers head waves develop, which travel horizontally along the top of the faster surface. Along a given array profile multiple headwave arrivals can occur and be measured. This method has been applied at scales from 1000 km, e.g. the Soviet “peaceful nuclear weapons” program which imaged lithospheric scale structure, to arrays of 30 m to investigate shallow soil and rock characteristics. There are numerous engineering, resource exploration and environmental applications.

## Objective 
In this exercise, we will investigate the different seismic refraction record sections from the experiment.

In [1]:
#Initial Setup and Subroutine Definitions
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

## Exercises
Several years ago a geotechnical engineering consulting company carried out a seismic refraction experiment over a graded building pad to investigate the subsurface velocity structure for forensics of the grading work. Figure 1 illustrates the site. The 24 channel seismic system had sensors deployed at even 5 ft intervals. The seisemic source consisted of repeated 20lb hammer blows (3x) on an aluminum striking plate. The team could move the source position around to generate different record sections. Figure 2 shows one of the 24-channel record sections, which contains data from zero time to 0.160 seconds, by way of an example. 

Figure 3 shows the same record section as Figure 2 adjusting the scale from 0 to 0.06 seconds to make the first arrivals clearer and easier to pick. Figure 4, and 5 show record sections from the same array of geophones for different shot points, shown in red. You will use the record sections Figures 3-5 to make first arrival time measurements, estimate the apparent velocities of direct and refracted arrivals, and to invert the data for a dipping plane-layer model.

##### Figure 1. Photograph and topography map of the site. The photograph is view east, and the topographic map is oriented with north at the top of the page.

<img src='fig1a.png'>
<img src='fig1b.png'>

##### Figure 2. An example survey record section from one shot. There is a very clear refracted arrival beyond geophone 8. Later phases are secondary P-waves and surface waves.

<img src='fig2.jpg'>


##### Figure 3 Seismic record section for source position located 5 ft south of the array. This figure limits the time axis from zero to 0.06 seconds for the purposes of your analysis. The minor grid ticks are 1ms.

<img src='fig3_grid.png'>

##### Figure 4 Seismic record section for source position located 5 ft north of the array.
<img src='fig4_grid.png'>

##### Figure 5 Seismic record section for source position located between geophones 12 and 13 
<img src='fig5_grid.png'>

### Exercise 1: In Class Activity

We will work with Figures 3 & 4 during class time. Using digital tools or printed copies and a ruler, we'll draft in straight lines fitted to the arrivals and identify where changes in slope occur. We'll tabulate the arrival time results for a two-layer model and use them for the remaining exercises.

<!-- BEGIN QUESTION -->

#### Question 1.1
Add the annotated figures we produced during class to the cell below by uploading them to the same folder as this notebook and adding a line like `<img src=filename.png>`. Are there any deviations from the assumption of a straight line fit to the refracted arrivals?

<!--
BEGIN QUESTION
name: q1.1
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

#### Question 1.2
Enter the arrival times from class in the arrays below.

<!--
BEGIN QUESTION
name: q1.2
-->

In [2]:
#Enter the Distance Vector (ft)
x = np.array([0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 65, 70, 75, 80, 90, 95, 100, 105, 110, 115, 120, 125, 130])

#Enter the Measured Arrival Times for forward line (milliseconds) as np.array
# t1 = np.array([0, 4.4, 9.1, 12.8, 15.6, 19.9, 24.2, 28.7, 32.7, 34.4, 33.5, 33.9, 33.8, 34.5, 36.6, 36.8, 36.9, 37.9, 38.5, 39.0, 40.0, 41.8, 42.1, 42.6, 43.0]) # SOLUTION
t1 = ...

#Enter the Measured Arrival Times (increasing) for reverse line (milliseconds) as np.array
# t2 = np.array([0, 3.7, 5.5, 8.2, 11.0, 13.4, 14.2, 15.5, 17.9, 19.0, 20.7, 22.0, 24.4, 26.3, 29.3, 32.2, 33.9, 36.9, 38.9, 39.8, 40.6, 40.61, 41.9, 42.1, 43.5]) # SOLUTION
t2 = ...


In [ ]:
grader.check("q1.2")

#### Question 1.3
Estimate the apparent velocities for the direct and head waves **in ft/s** from the forward and reverse shots. Note that the direct velocities should approximately match. You might be surprised by how many (or few) arrivals belong to the direct wave.

<!--
BEGIN QUESTION
name: q1.3
-->

In [4]:
# Forward line (Fig. 3) 
v_direct_forward = ...
v_head_forward = ...

# Reverse line (Fig. 4)
v_direct_reverse = ...
v_head_reverse = ...

In [ ]:
grader.check("q1.3")

### Exercise 2: Dipping Layers
The arrivals for the two off end shots (Figures 3 & 4) are not symmetric. We learned in lecture that this indicates a dipping layer. Figure 6 gives the geometry and equations for the two-layer problem. These equations are translated to python code in the cells below.

Note: $T$ are the intercept times. $i$ are the angle of incidence, $\alpha$ is the dip of the layer, and $V_0$, $V_{1d}$, $V_{1u}$, and $V_1$ are the velocities of the upper layer (average from all three profiles), the down dip apparent velocity of the deeper layer, the up dip apparent velocity of the deeper layer, and the actual velocity of the deeper layer. The down dip apparent velocity, $V_{1d}$, is less than the up dip apparent velocity of the refractor.

<img src='fig6.jpg'>

In [6]:
# review but don't edit this function
def seisrefract(x,t,fr,w,c):
    '''
    Program to compute and plot model velocities from array geometry traveltime picks.
    
    Inputs:
      x  : distance vector in feet [np.array]
      t  : measured arrival time picks in seconds [np.array]
      fr : 1=forward; 0=reverse designation of line [int]
      w  : velocity model layer designation [np.array] 
                1 = velocity1 in the velocity model, 2 = velocity2, etc.
                0 = station not used
      c  : color of line to plot [str]
      
    Outputs:
      t0 : intercept times
      p  : slopes (slownesses)
      errors: fit errors for all stations used in seconds
      
    '''
    
    # set some initial startup variables
    numline = max(w) # determine how many slopes to fit
    zero = np.where(w==0)[0] # find excluded points
    zflag = len(zero) # count excluded points
    t0 = []
    p = []
    errors = []
    
    # prepare full [1 x] matrix for regression
    tmp = np.ones(np.size(x))
    X = np.column_stack([tmp.T, x.T])
    
    # for-loop over each velocity in the model
    for J in np.arange(1, numline+1, 1):
        
        # Find Values to use in regressions
        k = np.where(w==J)[0] # find points for this velocity layer

        # Least-squares solution because matrix is non-square
        # solves for intercept and slope of linear fit to currently selected arrivals
        a = np.linalg.lstsq(X[k], t[k].T, rcond=None)[0]
        t0.append(a[0]) # save intercept time
        p.append(a[1]) # save slope
        linex_J = x[k] # take x points for fit line
        linet_J = np.sum(a*X[k],axis=1) # calculate fit line by summing t0 + p*x for each point
        errors.extend(list(linet_J - t[k])) # store errors for final RMSE calculation

        # Plotting
        if fr == 1:
            # forward line has earliest arrivals at shortest distances, no plot adjustment
            plt.plot(linex_J,linet_J,'-',color=c)
            plt.plot(x,t,'o',color=c,markerfacecolor='w',markersize=10)
        elif fr == 0:
            # reverse line has earlisest arrivals at longest distances, need to flip plot
            plt.plot(max(x)-linex_J,linet_J,'-',color=c)
            plt.plot(abs(x-max(x)),t,'o',color=c,markerfacecolor='w',markersize=10)
    
    # Remaining Plotting
    # X-out the measurements not assigned to a velocity model (and not used in the inversion)
    if(zflag > 0):
        if fr ==1:
            plt.plot(x[zero],t[zero],marker='+',lw=0,color='k')
        elif fr ==0:
            plt.plot(abs(x[zero]-max(x)),t[zero],marker='+',lw=0,color='k')
            
    # Add labels
    plt.title('Traveltime Plot',fontsize=20)
    plt.xlabel('Distance (ft)',fontsize=18)
    plt.ylabel('Time (sec)',fontsize=18)
    plt.tight_layout()
    
    return t0, p, errors

<!-- BEGIN QUESTION -->

#### Question 2.1
Using the arrival times tabulated in class for the forward and reverse off-end shots, perform an inversion for the plane-layered velocity model with one dipping layer (two layers total). Copy the time arrays you entered in Question 1.2 below for clarity. Fill in the w1 and w2 arrays by assigning the each arrival time in t1 and t2 to either the upper (1) or lower/dipping (2) layer. A plausible choice for w1 is given as an example but feel free to adjust it. You can exclude an arrival you consider an outlier by assigning it a 0 in the appropriate w-array spot.

Use all three of the following cells to fit and plot slopes, solve and plot the layer model, and view tabulated results.

<!--
BEGIN QUESTION
name: q2.1
manual: true
-->

In [7]:
# Enter the Distance Vector (ft)
x= np.array([0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 65, 70, 75, 80, 90, 95, 100, 105, 110, 115, 120, 125, 130])

# Re-enter the Measured Arrival Times for forward line (milliseconds) as np.array
t1 = ...

# Re-enter the Measured Arrival Times (increasing) for reverse line (milliseconds) as np.array
t2 = ...

# Entered measurements above should be in units of (milliseconds), but we need sec. Multiply by this factor...
timebase=0.001; 
t1 = timebase*t1
t2 = timebase*t2

# Enter the curve designator for forward line
# Note: len(w1) == len(t1)
w1=np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

# Enter the curve designator for reverse line
# Note: The first layer slopes should match, as in Fig. 6
#       Recall how many/few points fit the direct arrival in Question 1.3
w2= ...

# Execute seisrefract for forward and reverse parameters
plt.figure(figsize=(8,8))
t01,p1,e1 = seisrefract(x,t1,1,w1,'blue')
t02,p2,e2 = seisrefract(x,t2,0,w2,'red')

# Calculate total RMSE
RMSE = np.sqrt(np.sum(np.concatenate([e1,e2])**2))
print(f'RMSE: {RMSE*1000:.1f} ms')

<!-- END QUESTION -->

In [8]:
# Combine seisrefract results from forward and reverse lines to 
#   interpret and plot the dipping layer(s) model

numline=np.max(w1)
m=np.size(x)
V0=(1/p1[0] + 1/p2[0])/2; # Find average upper velocity
V0a=1/p1[0]; # Check forward upper velocity
V0b=1/p2[0]; # and reverse upper velocity, ideally they match
T0a=t01[0]; # First layer intercepts should be ~0
T0b=t02[0];
V1a=1/p1[1]; # Forward apparent velocity of head wave
V1b=1/p2[1]; # Reverse "
T1a=t01[1]; # Forward intercept time
T1b=t02[1]; # Reverse "

# V1b < V1a, so we use V1b = V1d and V1a = V1u for the Eqs. above
i1=0.5*(np.arcsin(V0/V1b)+np.arcsin(V0/V1a)); # critical angle
o1=0.5*(np.arcsin(V0/V1b)-np.arcsin(V0/V1a)); # angle of dipping layer

V1=V0/np.sin(i1); # lower layer velocity

# Calculate depth to dipping layer at each end
z1a=V0*T1a/(2*np.cos(i1)); 
z1b=V0*T1b/(2*np.cos(i1));
h1a=z1a/np.cos(o1);
h1b=z1b/np.cos(o1);
    
fig,ax = plt.subplots(1,1,figsize=(8,8))

# Add labels
plt.title('Velocity Cross Section',fontsize=20)
plt.xlabel('distance (ft)',fontsize=18)
plt.ylabel('depth (ft)',fontsize=18)
plt.tight_layout()

dx = 5
xx=np.arange(x[0], x[len(x)-1]+dx, dx);
yy=np.linspace(h1a, h1b, len(xx));
plt.plot(xx,yy,'m')
ax.text((x[m-1]-x[1])/2,h1a/2,'%.02f ft/sec' % V0,fontsize=20)
ax.text((x[m-1]-x[1])/2,(h1a+h1b)/2+15,'%.02f ft/sec' % V1,fontsize=20)

plt.ylim([max(yy)*3,0])
plt.xlim([0,max(x)])

In [9]:
#Write results to screen
print('V0a = %.0f  T0a = %f  V0b = %.0f  T0b = %f\n' % (V0a,T0a,V0b,T0b))
print('V1a=%.0f  T1a=%f  V1b=%.0f  T1b=%f\n' % (V1a,T1a,V1b,T1b))
print('Fit = %.1f ms' % (RMSE*1000))
print('\n\n')
print('Depth(ft)    Depth(ft)     Velocity(ft/sec)\n')
print('   0            0             %.0f\n' % (V0))
print('  %.1f         %.1f            %.0f\n' % (h1a, h1b, V1))

**SOLUTION NO PROMPT**: check model

<!-- BEGIN QUESTION -->

#### Question 2.2
Now consider a model with two dipping (three total) layers by changing some of the last points in each w-array to 3s.

<!--
BEGIN QUESTION
name: q2.2
manual: true
-->

In [10]:
# Enter the Distance Vector (ft)
x= np.array([0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 65, 70, 75, 80, 90, 95, 100, 105, 110, 115, 120, 125, 130])

# Enter the Measured Arrival Times for forward line (milliseconds) as np.array
t1 = ...

# Re-enter the Measured Arrival Times (increasing) for reverse line (milliseconds) as np.array
t2 = ...

# Entered measurements above should be in units of (milliseconds), but we need sec. Multiply by this factor...
timebase=0.001; 
t1 = timebase*t1
t2 = timebase*t2

# Enter the curve designator for forward line
# Note: len(w1) == len(t1)
w1= ...

# Enter the curve designator for reverse line
# Note: The first layer slopes should match, as in Fig. 6
#       Recall how many/few points fit the direct arrival in Question 1.3
w2= ...

# Execute seisrefract for forward and reverse parameters
plt.figure(figsize=(8,8))
t01,p1,e1 = seisrefract(x,t1,1,w1,'blue')
t02,p2,e2 = seisrefract(x,t2,0,w2,'red')

# Calculate total RMSE
RMSE = np.sqrt(np.sum(np.concatenate([e1,e2])**2))
print(f'RMSE: {RMSE*1000:.1f} ms')

<!-- END QUESTION -->

In [11]:
# Combine seisrefract results from forward and reverse lines to 
#   interpret and plot the dipping layer(s) model

numline=np.max(w1)
m=np.size(x)
V0=(1/p1[0] + 1/p2[0])/2; # Find average upper velocity
V0a=1/p1[0]; # Check forward upper velocity
V0b=1/p2[0]; # and reverse upper velocity, ideally they match
T0a=t01[0]; # First layer intercepts should be ~0
T0b=t02[0];
V1a=1/p1[1]; # Forward apparent velocity of head wave
V1b=1/p2[1]; # Reverse "
T1a=t01[1]; # Forward intercept time
T1b=t02[1]; # Reverse "

# V1b < V1a, so we use V1b = V1d and V1a = V1u for the Eqs. above
i1=0.5*(np.arcsin(V0/V1b)+np.arcsin(V0/V1a)); # critical angle
o1=0.5*(np.arcsin(V0/V1b)-np.arcsin(V0/V1a)); # angle of dipping layer

V1=V0/np.sin(i1); # lower layer velocity

# Calculate depth to dipping layer at each end
z1a=V0*T1a/(2*np.cos(i1)); 
z1b=V0*T1b/(2*np.cos(i1));
h1a=z1a/np.cos(o1);
h1b=z1b/np.cos(o1);

#add Next Layer if using a 3-layer velocity model
if(numline == 3):

    V2a=1/p1[2];
    V2b=1/p2[2];
    T2a=t01[2];
    T2b=t02[2];

    b21=np.arcsin(V0/V2a)+o1;
    a21=np.arcsin(V0/V2b)-o1;
    g21=np.arcsin(V1/V0*np.sin(a21));
    d21=np.arcsin(V1/V0*np.sin(b21));
    i2=(g21+d21)/2;
    o2=(g21-d21)/2 - o1;
    V2=V1/np.sin(i2);

    z2a=V1*(T2a-z1a/V0*(np.cos(a21+b21)+1)/np.cos(a21))/2/np.cos(i2);
    z2b=V1*(T2b-z1b/V0*(np.cos(a21+b21)+1)/np.cos(b21))/2/np.cos(i2);

    h2a=1/np.cos(o2)*(z1a*np.cos(a21-o2+o1)/np.cos(a21)+z2a);
    h2b=1/np.cos(o2)*(z1b*np.cos(b21+o2-o1)/np.cos(b21)+z2b);
    
fig,ax = plt.subplots(1,1,figsize=(8,8))

# Add labels
plt.title('Velocity Cross Section',fontsize=20)
plt.xlabel('distance (ft)',fontsize=18)
plt.ylabel('depth (ft)',fontsize=18)
plt.tight_layout()

dx = 5
xx=np.arange(x[0], x[len(x)-1]+dx, dx);
yy=np.linspace(h1a, h1b, len(xx));
plt.plot(xx,yy,'m')
ax.text((x[m-1]-x[1])/2,h1a/2,'%.02f ft/sec' % V0,fontsize=20)
ax.text((x[m-1]-x[1])/2,(h1a+h1b)/2+15,'%.02f ft/sec' % V1,fontsize=20)

if(numline == 3):
    yy=np.linspace(h2a, h2b, len(xx));
    plt.plot(xx,yy,'b')
    ax.text((x[m-1]-x[2])/2,(h2a+h2b)/2+15,'%.02f ft/sec' % V2,fontsize=20)

plt.ylim([max(yy)*3,0])
plt.xlim([0,max(x)])

In [12]:
#Write results to screen
print('V0a = %.0f  T0a = %f  V0b = %.0f  T0b = %f\n' % (V0a,T0a,V0b,T0b))
print('V1a=%.0f  T1a=%f  V1b=%.0f  T1b=%f\n' % (V1a,T1a,V1b,T1b))
if (numline == 3):
    print('V2a=%.0f  T2a=%f  V2b=%.0f  T2b=%f\n' % (V2a,T2a,V2b,T2b))
print('Fit = %.1f ms' % (RMSE*1000))
print('\n\n')
print('Depth(ft)    Depth(ft)     Velocity(ft/sec)\n')
print('   0            0             %.0f\n' % (V0))
print('  %.1f         %.1f            %.0f\n' % (h1a, h1b, V1))
if(numline == 3):
    print('  %.1f         %.1f          %.0f\n' % (h2a, h2b, V2))

**SOLUTION NO PROMPT**: check model

### Exercise 3: Using Center Shot
Using the center shot it is possible to break the array into two parts to see if there is non-planar structure. In other words it is possible to perform the analysis using the southern shot and the south-arm of the center shot, and the northern shot, and the north-arm of the center shot separately.

<img src='fig5_grid.png'>

#### Question 3.1
Pick and enter the arrival times for the center shot. We're divinding the array into a South (left) half and a North (right) half, as labeled.

<!--
BEGIN QUESTION
name: q3.1
-->

In [13]:
# All arrays are now half as long
x = np.array([0, 2.5, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55])

# South array in milliseconds
tS = ...

# Nort array in milliseconds
tN = ...
# tester below will just check array length

# convert to seconds
timebase=0.001; 
tS = timebase*tS
tN = timebase*tN

In [ ]:
grader.check("q3.1")

<!-- BEGIN QUESTION -->

#### Question 3.2
Analyze the South half of the array using a single dipping layer.

<!--
BEGIN QUESTION
name: q3.2
manual: true
-->

In [15]:
# South array: South half of Fig. 3 is the forward shot, tS is the reverse shot
# Enter the curve designator for forward line (t1)
# Note: len(w1S) == len(tS)
w1S= ...

# Enter the curve designator for reverse line (tS)
w2S= ...

# Execute seisrefract for forward and reverse parameters
plt.figure(figsize=(8,8))
t01S,p1S,e1S = seisrefract(x,t1[:13],1,w1S,'blue')
t02S,p2S,e2S = seisrefract(x,tS,0,w2S,'red')
plt.title('Traveltime Plot: South Half-Array', fontsize=20);

# Calculate total RMSE
RMSE_S = np.sqrt(np.sum(np.concatenate([e1S,e2S])**2))
print(f'RMSE: {RMSE_S*1000:.1f} ms')

<!-- END QUESTION -->

In [16]:
# Combine seisrefract results from forward and reverse lines to 
#   interpret and plot the dipping layer(s) model

numline=np.max(w1S)
m=np.size(x)
V0=(1/p1S[0] + 1/p2S[0])/2; # Find average upper velocity
V0a=1/p1S[0]; # Check forward upper velocity
V0b=1/p2S[0]; # and reverse upper velocity, ideally they match
T0a=t01S[0]; # First layer intercepts should be ~0
T0b=t02S[0];
V1a=1/p1S[1]; # Forward apparent velocity of head wave
V1b=1/p2S[1]; # Reverse "
T1a=t01S[1]; # Forward intercept time
T1b=t02S[1]; # Reverse "

# V1b < V1a, so we use V1b = V1d and V1a = V1u for the Eqs. above
i1=0.5*(np.arcsin(V0/V1b)+np.arcsin(V0/V1a)); # critical angle
o1=0.5*(np.arcsin(V0/V1b)-np.arcsin(V0/V1a)); # angle of dipping layer

V1=V0/np.sin(i1); # lower layer velocity

# Calculate depth to dipping layer at each end
z1a=V0*T1a/(2*np.cos(i1)); 
z1b=V0*T1b/(2*np.cos(i1));
h1a=z1a/np.cos(o1);
h1b=z1b/np.cos(o1);
    
fig,ax = plt.subplots(1,1,figsize=(8,8))

# Add labels
plt.title('Velocity Cross Section: South',fontsize=20)
plt.xlabel('distance (ft)',fontsize=18)
plt.ylabel('depth (ft)',fontsize=18)
plt.tight_layout()

dx = 5
xx_S=np.arange(x[0], x[len(x)-1]+dx, dx);
yy_S=np.linspace(h1a, h1b, len(xx_S));
plt.plot(xx_S,yy_S,'m')
ax.text((x[m-1]-x[1])/2,h1a/2,'%.02f ft/sec' % V0,fontsize=20)
ax.text((x[m-1]-x[1])/2,(h1a+h1b)/2+15,'%.02f ft/sec' % V1,fontsize=20)

plt.ylim([max(yy_S)*3,0])
plt.xlim([0,max(x)])

In [17]:
#Write results to screen
print('V0a = %.0f  T0a = %f  V0b = %.0f  T0b = %f\n' % (V0a,T0a,V0b,T0b))
print('V1a=%.0f  T1a=%f  V1b=%.0f  T1b=%f\n' % (V1a,T1a,V1b,T1b))
print('Fit = %.1f ms' % (RMSE_S*1000))
print('\n\n')
print('Depth(ft)    Depth(ft)     Velocity(ft/sec)\n')
print('   0            0             %.0f\n' % (V0))
print('  %.1f         %.1f            %.0f\n' % (h1a, h1b, V1))

**SOLUTION NO PROMPT**: check model

<!-- BEGIN QUESTION -->

#### Question 3.3
Analyze the North half of the array using a single dipping layer.

<!--
BEGIN QUESTION
name: q3.3
manual: true
-->

In [18]:
# North array: tN is the forward shot, North half of Fig. 4 is the reverse shot
# Enter the curve designator for forward line (tN)
# Note: len(w1N) == len(tN)
w1N= ...

# Enter the curve designator for reverse line (t2)
w2N= ...

# Execute seisrefract for forward and reverse parameters
plt.figure(figsize=(8,8))
t01N,p1N,e1N = seisrefract(x,tN,1,w1N,'blue')
t02N,p2N,e2N = seisrefract(x,t2[:13],0,w2N,'red')
plt.title('Traveltime Plot: North Half-Array', fontsize=20);

# Calculate total RMSE
RMSE_N = np.sqrt(np.sum(np.concatenate([e1N,e2N])**2))
print(f'RMSE: {RMSE_N*1000:.1f} ms')

<!-- END QUESTION -->

In [19]:
# Combine seisrefract results from forward and reverse lines to 
#   interpret and plot the dipping layer(s) model

numline=np.max(w1N)
m=np.size(x)
V0=(1/p1N[0] + 1/p2N[0])/2; # Find average upper velocity
V0a=1/p1N[0]; # Check forward upper velocity
V0b=1/p2N[0]; # and reverse upper velocity, ideally they match
T0a=t01N[0]; # First layer intercepts should be ~0
T0b=t02N[0];
V1a=1/p1N[1]; # Forward apparent velocity of head wave
V1b=1/p2N[1]; # Reverse "
T1a=t01N[1]; # Forward intercept time
T1b=t02N[1]; # Reverse "

# V1b < V1a, so we use V1b = V1d and V1a = V1u for the Eqs. above
i1=0.5*(np.arcsin(V0/V1b)+np.arcsin(V0/V1a)); # critical angle
o1=0.5*(np.arcsin(V0/V1b)-np.arcsin(V0/V1a)); # angle of dipping layer

V1=V0/np.sin(i1); # lower layer velocity

# Calculate depth to dipping layer at each end
z1a=V0*T1a/(2*np.cos(i1)); 
z1b=V0*T1b/(2*np.cos(i1));
h1a=z1a/np.cos(o1);
h1b=z1b/np.cos(o1);
    
fig,ax = plt.subplots(1,1,figsize=(8,8))

# Add labels
plt.title('Velocity Cross Section: North',fontsize=20)
plt.xlabel('distance (ft)',fontsize=18)
plt.ylabel('depth (ft)',fontsize=18)
plt.tight_layout()

dx = 5
xx_N=np.arange(x[0], x[len(x)-1]+dx, dx);
yy_N=np.linspace(h1a, h1b, len(xx_N));
plt.plot(xx_N,yy_N,'m')
ax.text((x[m-1]-x[1])/2,h1a/2,'%.02f ft/sec' % V0,fontsize=20)
ax.text((x[m-1]-x[1])/2,(h1a+h1b)/2+3,'%.02f ft/sec' % V1,fontsize=20)

plt.ylim([max(yy_N)*3,0])
plt.xlim([0,max(x)])

In [20]:
#Write results to screen
print('V0a = %.0f  T0a = %f  V0b = %.0f  T0b = %f\n' % (V0a,T0a,V0b,T0b))
print('V1a=%.0f  T1a=%f  V1b=%.0f  T1b=%f\n' % (V1a,T1a,V1b,T1b))
print('Fit = %.1f ms' % (RMSE_N*1000))
print('\n\n')
print('Depth(ft)    Depth(ft)     Velocity(ft/sec)\n')
print('   0            0             %.0f\n' % (V0))
print('  %.1f         %.1f            %.0f\n' % (h1a, h1b, V1))

**SOLUTION NO PROMPT**: check model

In [21]:
# Plot non-planar velocity model
fig,ax = plt.subplots(1,1,figsize=(8,8))

# Add labels
plt.title('Velocity Cross Section: Combined',fontsize=20)
plt.xlabel('distance (ft)',fontsize=18)
plt.ylabel('depth (ft)',fontsize=18)
plt.tight_layout()

plt.plot(xx_S,yy_S,'m')
plt.plot(xx_N+xx_S[-1],yy_N,'m')
# ax.text((x[m-1]-x[1])/2,h1a/2,'%.02f ft/sec' % V0,fontsize=20)
# ax.text((x[m-1]-x[1])/2,(h1a+h1b)/2+3,'%.02f ft/sec' % V1,fontsize=20)

plt.ylim([max(yy_S)*3,0])
plt.xlim([0,2*xx_S[-1]])

# total RMSE
print(f'RMSE South half: {RMSE_S*1000:.1f} ms')
print(f'RMSE North half: {RMSE_N*1000:.1f} ms')
RMSE = np.sqrt(np.sum(np.concatenate([e1N,e2N,e1S,e2S])**2))
print(f'RMSE combined: {RMSE*1000:.1f} ms')

### Exercise 4: Interpretation

<!-- BEGIN QUESTION -->

#### Question 4.1
Do the results of your center-shot analysis indicate a non-planar structure? What conclusions can you draw about the subsurface structure? Describe your observations of any deviation from the assumed plane-dipping model (straight line fit to the observations). What further information/analysis would help increase your ability to describe the structure?

<!--
BEGIN QUESTION
name: q4.1
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

#### Question 4.2
The traveltime measurements have uncertainties that propagate to the model parameter results. How robust are the results (velocities, angles, depths) to the timing uncertainty? You can answer this question qualitatively. Play around with the times and layer assignments a bit and describe what you find.

<!--
BEGIN QUESTION
name: q4.2
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

#### Question 4.3
What materials might be present in the layers you identified? You can search online for the seismic velocities of common earth materials. Careful with units!

<!--
BEGIN QUESTION
name: q4.3
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



# Submission

You should see two options for submission: The `eps130_export` and the `grader.export()` cells. Use whichever works for you. If all else fails, print to pdf from your browser.

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a pdf file for you to submit. **Please save before exporting!** The exporter will not see any unsaved changes to your notebook.

In [ ]:
!../eps130_export eps130_hw5_seismicRefraction_v2.1.ipynb

[Access your pdf here.](./eps130_hw5_seismicRefraction_v2.0.pdf)

Remember to check that you pdf shows your most recent work before submitting.

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()